## Filter exome bed files so they contain only SNPs with 2 alleles and non-duplicated variants

In [ ]:
[global]
# the output directory for generated files
parameter: cwd = path
# BED Plink files for exome data
parameter: bedfiles = paths
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Number of threads
parameter: numThreads = 1
# Software container option
parameter: container_lmm = 'statisticalgenetics/lmm:1.5'

In [ ]:
#Filter snps and remove duplicates with Plink2
[plink_1]
input: bedfiles, group_by=1
output: f'{cwd}/{_input:bn}.snps_nondups.bed'
task: trunk_workers = 1, walltime = '6h', mem = '20G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: container=container_lmm, expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    awk -F' ' '{print $4}' ${_input:n}.bim | sort |uniq -d > ${_output:nn}.dup_pos
    awk 'NR==FNR{a[$1]; next} ($4 in a) {print $2}' ${_output:nn}.dup_pos ${_input:n}.bim  > ${_output:nn}.dup_vars  
    plink2 \
      --bfile ${_input:n} \
      --snps-only just-acgt\
      --exclude ${_output:nn}.dup_vars\
      --threads ${numThreads} \
      --make-bed \
      --out ${_output:n} 

In [ ]:
#Merge all the filtered files with Plink1.9
[plink_2]
input: group_by = 'all'
output: f'{cwd}/ukb23155.filtered.merged.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '60G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: container=container_lmm, expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    echo -e ${' '.join([str(x)[:-4] for x in _input[1:]])} | sed 's/ /\n/g' > ${_output:n}.merge_list
    plink \
    --bfile ${_input[0]:n} \
    --merge-list ${_output:n}.merge_list \
    --make-bed \
    --out ${_output:n} \
    --threads ${numThreads} \
    --memory 60000

In [2]:
tpl_file=../farnam.yml
bedfiles=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
plink_sos=~/project/UKBB_GWAS_dev/workflow/plink_snps_only.ipynb
plink_sbatch=../output/$(date +"%Y-%m-%d")_plink_snps_only.sbatch
cwd=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/exome_files_snpsonly
job_size=1
numThreads=22
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm_v1_5.sif
plink_args="""plink
    --cwd $cwd 
    --bedfiles $bedfiles  
    --job_size $job_size
    --numThreads $numThreads
    --container_lmm $container_lmm
"""
    
sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $plink_sos \
    --to-script $plink_sbatch \
    --args "$plink_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam (index=0) is ignored due to saved signature
INFO: farnam output:   ../output/2021-01-08_plink_snps_only.sbatch
INFO: Workflow farnam (ID=1333be6e8ff392a0) is ignored with 1 ignored step.



In [ ]:
#!/bin/bash
files=$(ls -v *.dups_pos)
for i in $files; do
   wc -l $i >> dups_pos_counts.txt
done

files=$(ls -v *.dup_vars)
for i in $files; do
   wc -l $i >> dups_vars_counts.txt
done

files=$(ls -v ukb23155_c{1..22}_b0_v1.bim)
for i in $files; do
   wc -l $i >> bim_counts.txt
done

files=$(ls -v ukb23155_c{1..22}_b0_v1.snps_nondups.bim)
for i in $files; do
   wc -l $i >> bim_counts_snps_nondups.txt
done

paste bim_counts.txt dups_pos_counts.txt dups_vars_counts.txt > stats_vars_removed.txt
